In [1]:
# Project simuluated HP
# Import packages

In [1]:
# Import libraries
import pandas as pd
import geopandas as gpd
# Operations on geometries
import shapely
import numpy as np
from numpy import nan
import math
import glob 
import xarray as xr
import xarray
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import tensorflow as tf
import statsmodels.api as sm
import xgboost
import xgboost as xgb
import csv as csv
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn import linear_model
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import TimeSeriesSplit,GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor  #Perforing grid search
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn import datasets, metrics, model_selection
from yellowbrick.regressor import residuals_plot


import scipy
from scipy import stats
import os

import matplotlib.pyplot as plt

from scipy.stats import skew
from collections import OrderedDict

import _pickle as cPickle

import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

from utils_functions.utils_data import *
from utils_functions.utils_ml import *
from utils_functions.utils_RF import *
from utils_functions.utils_plot import *

/Users/noeliaotero/opt/anaconda3/envs/py39/lib/python3.9/site-packages/seaborn/apionly.py:9: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [2]:
# change paths if needed
PATH =  '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/outmodels/simulations_PREVAH/Classical_withdoi/models/'
PATH_pip =  '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/outmodels/simulations_PREVAH/Classical_withdoi/pipelines/'
PATH_OUT = '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/outmodels_standard_HISTORICAL//climate_rcp_simulations/RF_predsplit_CH2018_withdoi/'


In [3]:
# also select the covariates according to the model used

colnames = ['date', 't2m','prec', 'prec7D', 'prec15D', 'prec30D', 't2m7D', 't2m15D',
       't2m30D','PREVAH','PREVAH7D', 'PREVAH15D', 'PREVAH30D','doi']
Features = ['t2m','prec', 'prec7D', 'prec15D', 'prec30D', 't2m7D', 't2m15D',
       't2m30D','PREVAH','PREVAH7D', 'PREVAH15D', 'PREVAH30D','doi']

In [4]:
# test: remove temperature
#colnames = ['date', 
#           'PREVAH','PREVAH7D', 'PREVAH15D', 'PREVAH30D','doi']
#Features = ['PREVAH','PREVAH7D', 'PREVAH15D', 'PREVAH30D','doi']

In [5]:
# Load RCP projections
PATH_CH2018 = '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/data_CH2018_tomodel/'

os.makedirs(PATH_OUT, exist_ok = True)

In [6]:
names_pw = os.listdir(PATH_CH2018)

In [7]:
names_pw

['AET Leventina',
 'Emosson (ESA)',
 'Blenio (OFIBLE)',
 'Kraftwerke Mauvoisin AG',
 'KW Rheinfelden CH',
 'Electra-Massa (EM)']

In [8]:
# load the models from historical conditions

In [9]:
nfiles = os.listdir(PATH)
# only select randomforest (best model-generally)
# I will select "rf_Predefinesplit" --check also rf

In [10]:
rf_mod = []
names_mod = []
method = "rf_Predefinesplit"
for i in range(0, len(nfiles)):
    ifile = [method in i for i in nfiles] 
    if ifile[i]:
        #print(nfiles[i])
        with open(PATH + nfiles[i], 'rb') as fid:
            model = cPickle.load(fid)
        rf_mod.append(model)
        names_mod.append(nfiles[i].split("_")[0])
        

In [11]:
names_mod

['Blenio (OFIBLE)',
 'Electra-Massa (EM)',
 'AET Leventina',
 'KW Rheinfelden CH',
 'Kraftwerke Mauvoisin AG',
 'Emosson (ESA)']

In [12]:
# Open pipelines
import joblib
pipfiles = os.listdir(PATH_pip)
# sort the list as names_mod
n = [0,3,4,1,2,5]
pipfiles_list = [pipfiles[i] for i in n]
l_fpipelines = []
for i in range(0, len(pipfiles_list)): 
    #with open(PATH_pip + pipfiles[i], 'rb') as fid:
    pip = joblib.load(PATH_pip + pipfiles_list[i])
    l_fpipelines.append(pip)
    

In [13]:
pipfiles_list

['Blenio (OFIBLE).pkl',
 'Electra-Massa (EM).pkl',
 'AET Leventina.pkl',
 'KW Rheinfelden CH.pkl',
 'Kraftwerke Mauvoisin AG.pkl',
 'Emosson (ESA).pkl']

In [44]:
def get_predictions (PATH_CH2018, rf_model, nam_mod, Features, PATH_OUT, mypipeline = None):
    """Get the predictions based on the RF model from the historical period
       Args: PATH_CH2018: directory with the files
             nam_mod: name of the model/station"""

 
    basenames = os.listdir(PATH_CH2018 + nam_mod)
    name_ch = [os.path.splitext(x)[0] for x in basenames]

    
    x = readfiles(PATH_CH2018 + nam_mod)
    m = rf_model
    l_predictions = []
    
    for i in range(0, len(name_ch)):
        print(i)
      
        #print(x[i].head())
        #if (x[i].name_p.unique()[0] != nam_mod):
        #    raise Exception('Something is not matching!')
        df = x[i]
        
        df_new = df.rename({'discharge': 'PREVAH', 'dis7D': 'PREVAH7D', 'dis15D': 'PREVAH15D','dis30D': 'PREVAH30D'},axis='columns')
        df_new["date"] = pd.to_datetime(df_new["date"])
        # create the doi
        # add day of the year
        df_new['doi'] = df_new.date.dt.dayofyear
        
        df_ref = df_new[(df_new.date.dt.year >= 1981) &  (df_new.date.dt.year <= 2021)]
        df_proj_ref = df_ref[Features]
        df_proj= df_new[Features]
        # scale the data
        if mypipeline == None:
            # select the same period 1981-2021 to scale
            #fpipeline = prepareData(df_proj)
            fpipeline = prepareData(df_proj_ref)
            X_proj = fpipeline.fit_transform(df_proj)
        elif mypipeline == 'FullRCP':
            fpipeline = prepareData(df_proj)
            X_proj = fpipeline.fit_transform(df_proj)    
        else:
            fpipeline = mypipeline
            X_proj = fpipeline.transform(df_proj)
            
        # Get the predictions
        predictions = m.model.predict(X_proj)
        # add the predictions to the data frame
        df_new['predictions'] = predictions
        # save the predictions
        PATH_stat = PATH_OUTPUT + '/' + nam_mod + '/'
        os.makedirs(PATH_stat, exist_ok=True)
        print('save predictions for', name_ch[i])
        df_new.to_csv(PATH_stat + name_ch[i] + '.csv')
        
    l_predictions.append(df_new)
        
    return(l_predictions)

In [45]:
PATH_OUT

'/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/outmodels_standard_fullRCP//climate_rcp_simulations/RF_predsplit_CH2018_withdoi/'

In [46]:
# Get the predictions for each 
#test= get_predictions(PATH_CH2018, rf_mod[0], names_mod[0], PATH_OUT)
for station in range(0, len(names_mod)):
    print(names_mod[station])
   # get_predictions(PATH_CH2018, rf_mod[station], names_mod[station],Features, PATH_OUT, mypipeline = None)
    get_predictions(PATH_CH2018, rf_mod[station], names_mod[station],Features, PATH_OUT, mypipeline = l_fpipelines[station])
   
    #get_predictions(PATH_CH2018, rf_mod[station], names_mod[station], PATH_OUT, "FullRCP")
    

Blenio (OFIBLE)
0


KeyError: '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/outmodels_standard_fullRCP//climate_rcp_simulations/RF_predsplit_CH2018_withdoi/'